<a href="https://colab.research.google.com/github/ramanathanlab/SciFM24-Tutorial/blob/main/Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Using Agents to automate Biology Protocols via code generation**

If you are using the jupyter notebook locally, please run on Colab instead: https://colab.research.google.com/drive/1MLKC9usK-rVZv42exuPgRV398hIHAfcZ?usp=sharing

## **Problem:**
In biology, liquid-handling experiments can be tedious to complete (**ex.** serial dilution in PCR). To automate these processes, We can convert lab instructions into robot-compatible code to automate protocols (**ex.** use opentrons robot to handle liquid distribution processes)

## **Agents:**

In this case, an "action agent" (code-generating language model class) can help us generate code and refine it. We will focus on **one** action agent to keep things simple. This agent will help us generate code for thermocycling and refine it based on execution errors feedback.
* **Note:** Agents can be finnicky...sometimes it will struggle to generate the right code!


In [5]:
# install packages
!pip install --quiet langchain==0.1.13
!pip install --quiet opentrons==7.2.1
!pip install --upgrade --quiet langchain-google-genai==1.0.1
!pip install --upgrade --quiet  langchain-openai
!git clone https://github.com/Bri636/michigan_demo.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 623.7/623.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
from langchain.prompts import FewShotChatMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, SystemMessagePromptTemplate
from opentrons import protocol_api
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import MessagesPlaceholder
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.llms import HuggingFaceEndpoint
from dataclasses import dataclass
import os, textwrap, subprocess, re
from pathlib import Path
from langchain_openai import ChatOpenAI
from getpass import getpass

# API Keys:

Two options:

1.) Use GPT (Paid): https://platform.openai.com/account/api-keys

2.) Use Gemini (Free): https://ai.google.dev/

In [34]:
# grab api key
my_model = 'Gemini' #@param ['Gemini', 'GPT']
api_key = getpass("Enter your Google or GPT api key here: ")

Enter your Google or GPT api key here: ··········


In [35]:
if my_model=='Gemini':
  llm = GoogleGenerativeAI(model='gemini-pro',
                           google_api_key=api_key)
elif my_model=="GPT":
  llm = ChatOpenAI(model='gpt-4', openai_api_key=api_key)

In [36]:
@dataclass
class Templates():
  system:str=""
  few_shot:str=""
  code:str=""
  input:str=""

  @classmethod
  def collect_templates(cls, agent:str) -> dataclass:
    cls.system=cls.get_template(agent, 'system')
    cls.direction=cls.get_template(agent, 'direction')
    cls.code=cls.get_template(agent, 'code')
    return cls

  def get_template(agent:str, type:str) -> str:
    '''Helper function for downloading template types
    Parameters:
    ----------
    agent: ['action' or 'critic']
    type: ['code', 'direction', 'system']

    Outputs:
    --------
    LLM prompts as strings
    '''
    filepath=Path(f"/content/michigan_demo/{agent}_{type}.txt")
    with open(filepath, 'r') as file:
      prompt = file.read()
    return prompt

In [28]:
class LabWorker():
    def __init__(self, temperature=0, model_type=None, verbose=True):
        self.temperature = temperature
        self.model_type = model_type
        self.verbose=verbose
        if self.model_type=='Gemini':
            self.llm = GoogleGenerativeAI(model="gemini-pro",
                                        google_api_key=api_key,
                                        verbose=self.verbose)
        elif self.model_type=='GPT':
            self.llm = ChatOpenAI(model='gpt-4',
                                  api_key=api_key,
                                  verbose=self.verbose)

    @classmethod
    def initialize(cls, temperature:int=0, model_type:str='gemini',
                   verbose:bool=True, templates=None) -> object:
        """Creates an instance of the agent and initializes a chains"""
        agent = cls(temperature, model_type, verbose)
        agent.init_chain(templates)
        return agent

    def init_chain(self, templates:dataclass) -> None:
        """Initialized LLMChain based on agent_type [action, critic]"""
        system_prompt = SystemMessagePromptTemplate.from_template(templates.system)
        few_shot_prompt=(
            HumanMessagePromptTemplate.from_template(templates.direction)
            + AIMessagePromptTemplate.from_template(templates.code))
        history_prompt = MessagesPlaceholder(variable_name="chat_history")
        memory_gen = ConversationBufferWindowMemory(k=10, memory_key = "chat_history", return_messages = True)
        input_prompt = HumanMessagePromptTemplate.from_template("{user_input}")
        self.final_prompt = (
                      system_prompt
                      + few_shot_prompt
                      + history_prompt
                      + input_prompt
                      )
        self.chain = LLMChain(llm = self.llm,
                              prompt = self.final_prompt,
                              verbose = self.verbose,
                              memory = memory_gen)
        return None

In [16]:
class ActionAgent(LabWorker):
    def candidate_script_extractor(self, raw_edits: str) -> str:
      """Takes in raw re-edit output from GPT and returns cleaned script
      Parameters
      =========
      raw_edits: str
          str that contains raw code with comments output from GPT
      Output
      ========
      format_code: str
          str that contains formatted code that can be ran in cli
      """
      pattern = re.escape("```") + r"(.*?)" + re.escape("```")
      matches = re.search(pattern, raw_edits, flags=re.DOTALL)
      if matches is None:
          return raw_edits
      format_code = matches.group(1).strip()
      return format_code

    def simulate_opentrons(self, code: str, storage_path:str) -> str:
      """Helper function that simulates code in opentrons
      Parameters
      ==========
      code: str
          code generatred from code_generator chain
          as a str
      save_path: str
          save path as a str
      Output
      =========
      str:
          str for output error or output
      """
      code = textwrap.dedent(code)
      with open(storage_path, "w") as file:
          file.write(code)
      cli = ["opentrons_simulate", storage_path]
      result = subprocess.run(args=cli, capture_output=True, text=True)
      return result

    def refine(self, code: str, num_retries:int, storage_path:str) -> str:
      """Function that takes in code and iteratively refines it by saving as a file and running
      it against the cli for execution errors.
      Parameters
      ==========
      code: str
        code as a string
      num_retries: int
        number times to refine code
      storage_path: str
        file path to store temporary code
      Output
      ======
      tuple[str]:
        stores code and final message"""

      code = self.candidate_script_extractor(code)
      result = self.simulate_opentrons(code, storage_path)
      for i in range(num_retries):
          if result.returncode==0:
              break
          error_message = """The script you provided failed, and the execution error is described below. Please fix the error.
          \n\nError Message:\n\n""" + result.stderr
          raw_edit = self.chain.run({'user_input': error_message})
          code = self.candidate_script_extractor(raw_edit)
          result = self.simulate_opentrons(code, storage_path)
      success=result.returncode==0
      if success and i == 0:
          message = f"Your code did not need to be fixed:\n{result.stdout}"
      elif success:
          message = f"Your code fixed itself in {i} tries:\n{result.stdout}"
      else:
          message = f"Your code failed to fix itself in {i} times:\n{result.stderr}"
      return code, message


In [ ]:
action_template = Templates.collect_templates('action')
# action_template.input='''
# 1.) Load a themocycler module of type 'thermocyclerModuleV2'.
# 2.) Loop through the following set of sub-tasks in a loop 20 times:
#     a.) Set the thermocycler block temperature to 95 C, set hold time to 30 seconds, and set block max volume to 32 ul.
#     b.) Set the thermocycler block temperature to 57 C, set hold time to 30 seconds, and set block max volume to 32 ul.
#     c.) Set the thermocycler block temperature to 72 C, set hold time to 60 seconds, and set block max volume to 50 ul.
# '''

# pipetting input
action_template.input='''
1.) Start by loading the tiprack onto slot 1 of the Opentrons robot. Use 'opentrons_96_tiprack_300ul' labware for the tip rack.
2.) Load a reservoir of type 'nest_12_reservoir_15ml' in slot 2.
3.) Load a plate of type 'nest_96_wellplate_200ul_flat' in slot 3.
4.) Using the pipette, transfer 100 µL from location 'A1' in the loaded reservoir to wells of our loaded plate.
5.) Using the pipette transfer 100 µL from location 'A2' in the loaded reservoir to the first well in the ith column. Mix with 3 repetitions of 50 µL.
'''

In [ ]:
actionagent = ActionAgent.initialize(templates=action_template, model_type=my_model)
code = actionagent.chain.run({'user_input':action_template.input})
print(code)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a coding assistant that will convert lab protocol instructions into executable Opentrons scripts using the Opentrons library. You will also help debug code if the code you generate is incorrect. Here are some general guidlines: 

1.) Respond in the format: 
```
from opentrons import protocol_api

metadata={'apiLevel': '2.0'}
```
just include your code in this format and nothing else, or you will be punished.

2.) Always import "from opentrons import protocol_api" as well as the metadata at the start of your script. 

Human: 1.) Start by loading the tip rack onto slot 2 of the Opentrons robot. Use 'geb_96_tiprack_1000ul' labware for the tip rack.
2.) Load a plate of type 'armadillo_96_wellplate_200ul_pcr_full_skirt' onto slot 3.
3.) Load a pipette of type 'p1000_single'. Load it as a left mount. Use the tipracks from geb_96_tiprack_1000ul as the tiprack.
4.) Use the pipette that you have just loaded to pick up a

In [ ]:
code, result = actionagent.refine(code, 5, "/candidate_code.py")
print(result)

Your code did not need to be fixed:
Picking up tip from A1 of Opentrons OT-2 96 Tip Rack 300 µL on 1
Transferring 100.0 from A1 of NEST 12 Well Reservoir 15 mL on 2 to A1 of NEST 96 Well Plate 200 µL Flat on 3
	Aspirating 100.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 2 at 150.0 uL/sec
	Dispensing 100.0 uL into A1 of NEST 96 Well Plate 200 µL Flat on 3 at 300.0 uL/sec
	Aspirating 100.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 2 at 150.0 uL/sec
	Dispensing 100.0 uL into B1 of NEST 96 Well Plate 200 µL Flat on 3 at 300.0 uL/sec
	Aspirating 100.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 2 at 150.0 uL/sec
	Dispensing 100.0 uL into C1 of NEST 96 Well Plate 200 µL Flat on 3 at 300.0 uL/sec
	Aspirating 100.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 2 at 150.0 uL/sec
	Dispensing 100.0 uL into D1 of NEST 96 Well Plate 200 µL Flat on 3 at 300.0 uL/sec
	Aspirating 100.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 2 at 150.0 uL/sec
	Dispensing 100.0 uL into E1 of NEST 96 Well

In [ ]:
print(f'***Original Instructions:***\n{action_template.input}')
print(f'***Final Proposed Code:***\n\n{code}')

***Original Instructions:***

1.) Start by loading the tiprack onto slot 1 of the Opentrons robot. Use 'opentrons_96_tiprack_300ul' labware for the tip rack.
2.) Load a reservoir of type 'nest_12_reservoir_15ml' in slot 2.
3.) Load a plate of type 'nest_96_wellplate_200ul_flat' in slot 3.
4.) Using the pipette, transfer 100 µL from location 'A1' in the loaded reservoir to wells of our loaded plate. 
5.) Using the pipette transfer 100 µL from location 'A2' in the loaded reservoir to the first well in the ith column. Mix with 3 repetitions of 50 µL.

***Final Proposed Code:***

from opentrons import protocol_api

metadata={'apiLevel': '2.0'}

def run(protocol: protocol_api.ProtocolContext):
    tiprack = protocol.load_labware('opentrons_96_tiprack_300ul', 1)
    reservoir = protocol.load_labware('nest_12_reservoir_15ml', 2)
    plate = protocol.load_labware('nest_96_wellplate_200ul_flat', 3)

    pipette = protocol.load_instrument('p300_single', 'right', tip_racks=[tiprack])

    pipette

# General example you can play with:

**Note** We are hard coding this

System template: general "rules" or role that the GPT model can play as

few_shot_human: few shot example input for GPT model

few_shot_ai: few shot example output for GPT model

In [81]:
api_key = getpass()
llm_2 = ChatOpenAI(model='gpt-4', openai_api_key=api_key)
# if you want gemini-pro
# llm_2 = GoogleGenerativeAI(model='gemini-pro',
#                           google_api_key=api_key)


··········


In [82]:
system_template='''
You are a coding expert that can code really well in python. You will generate python code for me based on a set of directions specified by me.
'''

In [83]:
few_shot_human='''
Create a function that adds two numbers
'''

few_shot_ai='''
def add_two_numbers(a, b):
  return a + b
'''

In [84]:
system_prompt = SystemMessagePromptTemplate.from_template(system_template)
few_shot_prompt=(
    HumanMessagePromptTemplate.from_template(few_shot_human)
    + AIMessagePromptTemplate.from_template(few_shot_ai))
history_prompt = MessagesPlaceholder(variable_name="chat_history")
memory_gen = ConversationBufferWindowMemory(k=10, memory_key = "chat_history", return_messages = True)
input_prompt = HumanMessagePromptTemplate.from_template("{user_input}")
final_prompt = (
              system_prompt
              + few_shot_prompt
              + history_prompt
              + input_prompt
              )
chain = LLMChain(llm = llm_2,
                      prompt = final_prompt,
                      verbose = True,
                      memory = memory_gen)

In [85]:
code = chain.run({'user_input': "write a function that prints hello world"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a coding expert that can code really well in python. You will generate python code for me based on a set of directions specified by me.

Human: 
Create a function that adds two numbers

AI: 
def add_two_numbers(a, b): 
  return a + b

Human: write a function that prints hello world

> Finished chain.


In [86]:
print(code)

Sure, here is a simple function in python that prints "Hello World".

```python
def print_hello_world(): 
    print("Hello World")
```
